In [50]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import learn
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn import datasets, linear_model
from sklearn import cross_validation
import numpy as np
import pandas as pd
from pandas import Series

# np.set_printoptions(threshold=np.nan) # you can uncomment this to print the entire onehotlabels array
df = pd.read_csv("https://raw.githubusercontent.com/EnterpriseDeepLearning/housing-prices-wide-and-deep/master/train_validate.csv")
df = df.fillna(0) # this will probably come back to bite you
# print(df.dtypes) # the datatypes are floats, ints, and objects
# print(df.head(3)) # look at the beautiful data
dataset = df.values
# The code below sets floats to ints. This was originally written to easily select all categorical features,
    # here actually of type object, by 'excluding' those that are of type int. You can just use 'include' instead
# non_categorical_features = dataset[:, [1,2,3,4,5,7,8,11,12,16,18,24]] # select the non-categorical features
# for col in ['acres','tax_gross_amount','total_stories','assessment_value_town', 'garage_capacity', 
#'year_built', 'common_land_acres']:
    # df[col] = df[col].astype('int64') # set the type of all - manually selected - float64 to int64
# print(df.dtypes)
categorical_features = df.select_dtypes(include=[object])
#print(categorical_features.head(3)) # uncomment to to see the first three observations of the object, 
    # that is, categorical, features 
le = LabelEncoder() # abbreviated for label encoder
categorical_features_2 = categorical_features.apply(le.fit_transform)
enc = OneHotEncoder() # abbreviated for one hot encoder
enc.fit(categorical_features_2)
# 3. Transform
onehotlabels = enc.transform(categorical_features_2).toarray()
print("There are 15 categorical features:")
print(categorical_features_2.shape) 
print("This is the one hot encoding:")
print(onehotlabels)
print("There are 259 feature columns with the one hot encoding:")
print(onehotlabels.shape) 
# print(onehotlabels) # uncomment to see the first three observations of the transformed categorical features
    # you now have many more columns due to changing the categorical data into numerical data
# select and replace the categorical columns in the dataframe with the transformed columns
for col in ['address','city','garage_type','surveyed', 'seasonal', 'water_body_type', 'water_frontage_length',
    'short_sale', 'garage', 'flood_zone', 'easements', 'current_use', 'covenants', 'basement_access_type', 'basement']:
    df[col] = categorical_features_2[col]
dataset = df.values
x = dataset[:,0:27]
# print(x) # uncomment to see the fully processed feature columns from the dataset

print(x.shape)
labels = np.linspace(1, 206, 206, dtype=int)
y = dataset[:,27:28]
y=y.flatten()
# print(y.shape) # uncomment to see the output column
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(
x, y, test_size=0.2, random_state=42)

total_len = X_train.shape[0]
print(total_len)

There are 15 categorical features:
(206, 15)
This is the one hot encoding:
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  1.  1.  0.  0.  1.
   0.  0.  0.  0.  0.  1.  0.  0.  0.

In [14]:
# Parameters
learning_rate = 0.001
training_epochs = 500
batch_size = 10
display_step = 1
dropout_rate = 0.9
# Network Parameters
n_hidden_1 = 27 # 1st layer number of features
n_hidden_2 = 200 # 2nd layer number of features
n_hidden_3 = 200
n_hidden_4 = 256
n_input = X_train.shape[1]
n_classes = 1

# tf Graph input
x = tf.placeholder("float", [None, 27])
y = tf.placeholder("float", [None])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)

    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    # Hidden layer with RELU activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)

    # Hidden layer with RELU activation
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)

    # Output layer with linear activation
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], 0, 0.1)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], 0, 0.1)),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], 0, 0.1)),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4], 0, 0.1)),
    'out': tf.Variable(tf.random_normal([n_hidden_4, n_classes], 0, 0.1))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], 0, 0.1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], 0, 0.1)),
    'b3': tf.Variable(tf.random_normal([n_hidden_3], 0, 0.1)),
    'b4': tf.Variable(tf.random_normal([n_hidden_4], 0, 0.1)),
    'out': tf.Variable(tf.random_normal([n_classes], 0, 0.1))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)
tf.transpose(pred)

# Define loss and optimizer
cost = tf.reduce_mean(tf.square(pred-y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(total_len/batch_size)
        # Loop over all batches
        for i in range(total_batch-1):
            batch_x = X_train[i*batch_size:(i+1)*batch_size]
            batch_y = Y_train[i*batch_size:(i+1)*batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c, p = sess.run([optimizer, cost, pred], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch

        # sample prediction
        label_value = batch_y
        estimate = p
        err = label_value-estimate
        print ("num batch:", total_batch)

        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
            print ("[*]----------------------------")
            for i in xrange(3):
                print ("label value:", label_value[i], \
                    "estimated value:", estimate[i])
            print ("[*]============================")

    print ("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = sess.run(cost, feed_dict={x:X_test, y: Y_test})
    print(accuracy)
    predicted_vals = sess.run(pred, feed_dict={x: X_test})

Instructions for updating:
Use `tf.global_variables_initializer` instead.
num batch: 16
Epoch: 0001 cost= 622558692096.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 593064.75]
label value: 670000.0 estimated value: [ 446476.1875]
label value: 512500.0 estimated value: [ 526964.0625]
[*]============================
num batch: 16
Epoch: 0002 cost= 654025481728.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 368522.78125]
label value: 670000.0 estimated value: [ 277529.375]
label value: 512500.0 estimated value: [ 327454.4375]
[*]============================
num batch: 16
Epoch: 0003 cost= 586310162432.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 395036.09375]
label value: 670000.0 estimated value: [ 297575.15625]
label value: 512500.0 estimated value: [ 350900.9375]
[*]============================
num batch: 16
Epoch: 0004 cost= 591642683904.000000000
[*]----------------------

num batch: 16
Epoch: 0037 cost= 422443653504.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 538084.125]
label value: 670000.0 estimated value: [ 555025.25]
label value: 512500.0 estimated value: [ 551906.75]
[*]============================
num batch: 16
Epoch: 0038 cost= 421891520256.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 537904.875]
label value: 670000.0 estimated value: [ 556768.625]
label value: 512500.0 estimated value: [ 553081.6875]
[*]============================
num batch: 16
Epoch: 0039 cost= 420949545728.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 540519.375]
label value: 670000.0 estimated value: [ 560991.0625]
label value: 512500.0 estimated value: [ 556874.4375]
[*]============================
num batch: 16
Epoch: 0040 cost= 420059000448.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 541422.1875]
label value: 670000.

num batch: 16
Epoch: 0073 cost= 410278784896.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 534503.125]
label value: 670000.0 estimated value: [ 604826.375]
label value: 512500.0 estimated value: [ 585236.5625]
[*]============================
num batch: 16
Epoch: 0074 cost= 410015593728.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 536236.3125]
label value: 670000.0 estimated value: [ 607114.6875]
label value: 512500.0 estimated value: [ 587507.9375]
[*]============================
num batch: 16
Epoch: 0075 cost= 409825216768.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 535265.1875]
label value: 670000.0 estimated value: [ 607335.0625]
label value: 512500.0 estimated value: [ 587359.4375]
[*]============================
num batch: 16
Epoch: 0076 cost= 409517283712.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 537880.8125]
label value: 6

num batch: 16
Epoch: 0109 cost= 402612630272.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 536855.625]
label value: 670000.0 estimated value: [ 636892.5]
label value: 512500.0 estimated value: [ 610065.6875]
[*]============================
num batch: 16
Epoch: 0110 cost= 402238530560.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 537496.6875]
label value: 670000.0 estimated value: [ 637749.75]
label value: 512500.0 estimated value: [ 611011.875]
[*]============================
num batch: 16
Epoch: 0111 cost= 401970658048.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 538786.25]
label value: 670000.0 estimated value: [ 639451.375]
label value: 512500.0 estimated value: [ 612685.3125]
[*]============================
num batch: 16
Epoch: 0112 cost= 401005598720.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 545121.6875]
label value: 670000.0 

num batch: 16
Epoch: 0145 cost= 396552227840.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 537249.6875]
label value: 670000.0 estimated value: [ 650741.]
label value: 512500.0 estimated value: [ 622657.0625]
[*]============================
num batch: 16
Epoch: 0146 cost= 396700802304.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 539290.75]
label value: 670000.0 estimated value: [ 652852.25]
label value: 512500.0 estimated value: [ 624823.875]
[*]============================
num batch: 16
Epoch: 0147 cost= 397306025216.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 536470.375]
label value: 670000.0 estimated value: [ 649731.]
label value: 512500.0 estimated value: [ 621926.5]
[*]============================
num batch: 16
Epoch: 0148 cost= 395339682560.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 557942.25]
label value: 670000.0 estimated

...

num batch: 16
Epoch: 0324 cost= 382542505472.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 564550.0625]
label value: 670000.0 estimated value: [ 691483.875]
label value: 512500.0 estimated value: [ 669367.25]
[*]============================
num batch: 16
Epoch: 0325 cost= 383252324608.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 551596.875]
label value: 670000.0 estimated value: [ 682062.875]
label value: 512500.0 estimated value: [ 657239.6875]
[*]============================
num batch: 16
Epoch: 0326 cost= 387077101824.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 539912.625]
label value: 670000.0 estimated value: [ 666693.25]
label value: 512500.0 estimated value: [ 643211.75]
[*]============================
num batch: 16
Epoch: 0327 cost= 387899031296.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 554417.0625]
label value: 670000.0 

num batch: 16
Epoch: 0360 cost= 382111073280.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 553325.0625]
label value: 670000.0 estimated value: [ 683142.125]
label value: 512500.0 estimated value: [ 659890.0625]
[*]============================
num batch: 16
Epoch: 0361 cost= 384779479296.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 541589.0625]
label value: 670000.0 estimated value: [ 670883.6875]
label value: 512500.0 estimated value: [ 647005.3125]
[*]============================
num batch: 16
Epoch: 0362 cost= 386518408448.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 549832.0625]
label value: 670000.0 estimated value: [ 676634.75]
label value: 512500.0 estimated value: [ 654861.1875]
[*]============================
num batch: 16
Epoch: 0363 cost= 384040564224.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 567738.3125]
label value: 67

num batch: 16
Epoch: 0396 cost= 384237604352.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 542845.75]
label value: 670000.0 estimated value: [ 674441.75]
label value: 512500.0 estimated value: [ 649619.875]
[*]============================
num batch: 16
Epoch: 0397 cost= 384979085824.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 548362.625]
label value: 670000.0 estimated value: [ 679864.5625]
label value: 512500.0 estimated value: [ 656285.125]
[*]============================
num batch: 16
Epoch: 0398 cost= 382478002432.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 569486.875]
label value: 670000.0 estimated value: [ 698456.625]
label value: 512500.0 estimated value: [ 677550.125]
[*]============================
num batch: 16
Epoch: 0399 cost= 379631576320.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 577128.]
label value: 670000.0 esti

num batch: 16
Epoch: 0432 cost= 377293853184.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 575216.]
label value: 670000.0 estimated value: [ 710855.25]
label value: 512500.0 estimated value: [ 689664.4375]
[*]============================
num batch: 16
Epoch: 0433 cost= 377106565120.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 571495.625]
label value: 670000.0 estimated value: [ 707125.875]
label value: 512500.0 estimated value: [ 685081.0625]
[*]============================
num batch: 16
Epoch: 0434 cost= 378244479232.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 570438.3125]
label value: 670000.0 estimated value: [ 705952.5625]
label value: 512500.0 estimated value: [ 683435.8125]
[*]============================
num batch: 16
Epoch: 0435 cost= 378439308288.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 567548.0625]
label value: 670000.

num batch: 16
Epoch: 0468 cost= 373340888064.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 566897.125]
label value: 670000.0 estimated value: [ 703041.]
label value: 512500.0 estimated value: [ 677805.]
[*]============================
num batch: 16
Epoch: 0469 cost= 376986693376.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 563643.625]
label value: 670000.0 estimated value: [ 704977.125]
label value: 512500.0 estimated value: [ 678813.25]
[*]============================
num batch: 16
Epoch: 0470 cost= 377369375232.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 570082.875]
label value: 670000.0 estimated value: [ 704930.5]
label value: 512500.0 estimated value: [ 680055.75]
[*]============================
num batch: 16
Epoch: 0471 cost= 376568742656.000000000
[*]----------------------------
label value: 1150000.0 estimated value: [ 575080.6875]
label value: 670000.0 estimated